In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob

In [2]:
elements = pmg.Element('O'), pmg.Element('Li'), pmg.Element('Ni')
o, li, ni = elements

charges = {
    li: 1,
    ni: 3,
    o: -2,
}

bpp_def = {
'O_O_A' : 22764.7000,
'O_O_rho' : 0.1490,
'O_O_C' : 20.765,
'O_Li_A' : 632.1018,
'O_Li_rho' : 0.2906,
'O_Li_C' : 0.0,
'O_Ni_A' : 1273.669,
'O_Ni_rho' : 0.333,
'O_Ni_C' : 0.0}

bpp = {
'O_O_A' : 22764.7000,
'O_O_rho' : 0.1490,
'O_O_C' : 20.765,
'O_Li_A' : 632.1018,
'O_Li_rho' : 0.2906,
'O_Li_C' : 0.0,
'O_Ni_A' : 1273.669,
'O_Ni_rho' : 0.333,
'O_Ni_C' : 0.1}

pairs = [(o,o), (o,ni), (o,li)]
structures = []
for structure in glob.glob('POSCAR*'):
    st = pmg.Structure.from_file(structure)
    st.remove_oxidation_states()
    structures.append(pmg.symmetry.analyzer.SpacegroupAnalyzer(st).get_conventional_standard_structure())

In [3]:
def init(structure):
    lmp = lammps.Lammps(units='electron', 
                    style = 'full',
                    args=['-log', 'none', '-screen', 'none'])
    lmp.system.add_pymatgen_structure(structure, elements)
    lmp.command('kspace_style pppm 1e-6')

    lmp.command('pair_style buck/coul/long 12.0')
    lmp.command('pair_coeff * * 0 1 0')

    

    for c in charges:
        lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))
    
    return lmp

In [4]:
def update_potentials(**kwargs):
    bpp.update(kwargs)
def set_potentials(instance):
    for pair in pairs:

        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]), 
                                      dex(pair[1]),  
                                      bpp['{}_{}_A'.format(pair[0], pair[1])], 
                                      bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                      bpp['{}_{}_C'.format(pair[0], pair[1])]
                                     )
             )
        
def dex(elin): return elements.index(elin) + 1

In [5]:
def simfunc(**kwargs):
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        
        out = np.zeros([instances[0].system.total,3, len(instances)])
        
        
        for instance in instances:
            set_potentials(instance)
            instance.run(0)
            out[:,:,instances.index(instance)] = instance.system.forces
            
            
            
    else: out = np.ones([instances[0].system.total,3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
    return out

In [6]:
instances = [init(structure) for structure in structures]

In [8]:
expected = np.zeros([instances[0].system.total,3, len(instances)])

IndexError: list index out of range

In [ ]:
with pm.Model() as model:
    O_O_C      = pm.Normal("O_O_C",    mu = bpp_def['O_O_C'],    sd =  5)
    O_Li_A     = pm.Normal("O_Li_A",   mu = bpp_def['O_Li_A'],   sd =  100)
    O_Li_rho   = pm.Normal("O_Li_rho", mu = bpp_def['O_Li_rho'], sd =  0.1)
    O_Mn_A     = pm.Normal("O_Ni_A",   mu = bpp_def['O_Ni_A'],   sd =  50)
    O_Mn_rho   = pm.Normal("O_Ni_rho", mu = bpp_def['O_Ni_rho'], sd =  0.05)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=1), draws=100)